In [1]:
#importing all the required libraries
%pylab inline

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
import plotly
import plotly.offline as pyoff
import plotly.graph_objs as go

np.random.seed(42)
import random
random.seed(42)
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
from datetime import datetime
import gc
import itertools

import lifetimes
from lifetimes import BetaGeoFitter
from lifetimes.plotting import *
from lifetimes.utils import *
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

Populating the interactive namespace from numpy and matplotlib


In [87]:
customers_db = pd.read_csv("olist_customers_dataset_new.csv")
order_items_db = pd.read_csv("olist_order_items_dataset.csv")
order_payments_db = pd.read_csv("olist_order_payments_dataset.csv")
orders_db = pd.read_csv("olist_orders_dataset_new.csv")
products_db = pd.read_csv("olist_products_dataset_new.csv")
sellers_db = pd.read_csv("olist_sellers_dataset_new.csv")

In [5]:
orders_db

,order_id2,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,ON34305,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,ON12238,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,ON21442,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,ON36041,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ON16462,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...,...
99436,ON61248,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,ON21623,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,ON26513,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,ON14010,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


In [7]:
order_payments_db

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54


In [13]:
order_payments_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
order_id                103886 non-null object
payment_sequential      103886 non-null int64
payment_type            103886 non-null object
payment_installments    103886 non-null int64
payment_value           103886 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [8]:
newdf = orders_db.merge(order_payments_db, on='order_id',how='right')


In [10]:
newdf

,order_id2,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,payment_value
0,ON34305,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,credit_card,1,18.12
1,ON34305,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,3,voucher,1,2.00
2,ON34305,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2,voucher,1,18.59
3,ON12238,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,boleto,1,141.46
4,ON21442,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,credit_card,3,179.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103881,ON61248,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,1,credit_card,3,85.08
103882,ON21623,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,1,credit_card,3,195.00
103883,ON26513,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,1,credit_card,5,271.01
103884,ON14010,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,1,credit_card,4,441.16


In [12]:
newdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103886 entries, 0 to 103885
Data columns (total 13 columns):
order_id2                        103886 non-null object
order_id                         103886 non-null object
customer_id                      103886 non-null object
order_status                     103886 non-null object
order_purchase_timestamp         103886 non-null object
order_approved_at                103711 non-null object
order_delivered_carrier_date     101998 non-null object
order_delivered_customer_date    100754 non-null object
order_estimated_delivery_date    103886 non-null object
payment_sequential               103886 non-null int64
payment_type                     103886 non-null object
payment_installments             103886 non-null int64
payment_value                    103886 non-null float64
dtypes: float64(1), int64(2), object(10)
memory usage: 11.1+ MB


In [14]:
aa = newdf[['order_id2','order_id','payment_sequential','payment_type', 'payment_installments','payment_value']]

In [34]:
aa.to_csv('olist_order_payments_dataset_new.csv')

In [16]:
order_items_db

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [18]:
newdf2= orders_db.merge(order_items_db, on='order_id',how='right')

In [19]:
newdf2

,order_id2,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,ON34305,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,ON12238,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,ON21442,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,ON36041,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ON16462,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,ON21623,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,1,f1d4ce8c6dd66c47bbaa8c6781c2a923,1f9ab4708f3056ede07124aad39a2554,2018-02-12 13:10:37,174.90,20.10
112646,ON26513,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,1,b80910977a37536adeddd63663f916ad,d50d79cb34e38265a8649c383dcffd48,2017-09-05 15:04:16,205.99,65.02
112647,ON14010,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,1,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59
112648,ON14010,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,2,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59


In [21]:
products_db

,productid_2,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,P23996,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,P26436,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,P05410,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,P03253,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,P21582,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...,...
32946,P12196,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,P18029,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,P07866,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,P27808,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [23]:
newdf3= products_db.merge(newdf2, on='product_id',how='right')

In [26]:
newdf3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 24 columns):
productid_2                      112650 non-null object
product_id                       112650 non-null object
product_category_name            111047 non-null object
product_name_lenght              111047 non-null float64
product_description_lenght       111047 non-null float64
product_photos_qty               111047 non-null float64
product_weight_g                 112632 non-null float64
product_length_cm                112632 non-null float64
product_height_cm                112632 non-null float64
product_width_cm                 112632 non-null float64
order_id2                        112650 non-null object
order_id                         112650 non-null object
customer_id                      112650 non-null object
order_status                     112650 non-null object
order_purchase_timestamp         112650 non-null object
order_approved_at                112635 no

In [27]:
order_items_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
order_id               112650 non-null object
order_item_id          112650 non-null int64
product_id             112650 non-null object
seller_id              112650 non-null object
shipping_limit_date    112650 non-null object
price                  112650 non-null float64
freight_value          112650 non-null float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [29]:
newdf4 = newdf3[['order_id2','order_id','order_item_id','productid_2','product_id','seller_id','shipping_limit_date','price','freight_value']]

In [30]:
newdf4

,order_id2,order_id,order_item_id,productid_2,product_id,seller_id,shipping_limit_date,price,freight_value
0,ON86158,e17e4f88e31525f7deef66779844ddce,1,P23996,1e9e8ef04dbcff4541ed26657ea517e5,5670f4db5b62c43d542e1b2d56b0cf7c,2018-04-30 17:33:54,10.91,7.39
1,ON31875,5236307716393b7114b53ee991f36956,1,P26436,3aa071139cb16b67ca9e5dea641aaa2f,b561927807645834b59ef0d16ba55a24,2018-02-06 19:11:15,248.00,17.99
2,ON44137,01f66e58769f84129811d43eefd187fb,1,P05410,96bd76ec8810374ed1b65e291975717f,7b07b3c7487f0ea825fc6df75abd658b,2018-07-11 21:30:20,79.80,7.82
3,ON80049,143d00a4f2dde4e0364ee1821577adb3,1,P03253,cef67bcfe19066a932b7673e239eb23d,c510bc1718f0f2961eaa42a23330681a,2018-08-07 09:10:13,112.30,9.54
4,ON15014,86cafb8794cb99a9b1b77fc8e48fbbbb,1,P21582,9dc1a7de274444849c219cff195d0b71,0be8ff43f22e456b4e0371b2245e4d01,2018-04-17 01:30:23,37.90,8.29
...,...,...,...,...,...,...,...,...,...
112645,ON97480,59e88b7d3675e89aceaf86f372d3bc9a,2,P27808,83808703fc0706a22e264b9d75f04a2e,4324dd16853115efb0fd9d0d131ba6f4,2018-01-10 03:32:09,29.90,16.11
112646,ON20378,6e4465d771f02e4fe335225de3c6c043,1,P27808,83808703fc0706a22e264b9d75f04a2e,4324dd16853115efb0fd9d0d131ba6f4,2018-03-14 02:30:48,29.90,23.28
112647,ON20378,6e4465d771f02e4fe335225de3c6c043,2,P27808,83808703fc0706a22e264b9d75f04a2e,4324dd16853115efb0fd9d0d131ba6f4,2018-03-14 02:30:48,29.90,23.28
112648,ON20378,6e4465d771f02e4fe335225de3c6c043,3,P27808,83808703fc0706a22e264b9d75f04a2e,4324dd16853115efb0fd9d0d131ba6f4,2018-03-14 02:30:48,29.90,23.28


In [31]:
newdf55= sellers_db.merge(newdf4, on='seller_id',how='right')

In [32]:
newdf55

,sellerid_2,seller_id,seller_zip_code_prefix,seller_city,seller_state,order_id2,order_id,order_item_id,productid_2,product_id,shipping_limit_date,price,freight_value
0,S1347,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,ON52911,4a90af3e85dd563884e2afeab1091394,1,P29061,ffb64e34a37740dafb6c88f1abd1fa61,2017-08-25 20:50:19,106.2,9.56
1,S1347,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,ON47598,bc8a5de6abf5b14f98a6135a7fb46731,2,P31368,325a06bcce0da45b7f4ecf2797dd40e4,2017-09-05 12:50:19,10.8,2.42
2,S1347,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,ON69627,6d953888a914b67350d5bc4d48f2acab,1,P32849,f4621f8ad6f54a2e3c408884068be46d,2017-05-11 16:25:11,101.7,15.92
3,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,ON44120,7a95f5626df09176d8823cbce3d9db1e,1,P31854,55103cbbe0851d3a303a022cc4140628,2017-08-17 23:30:18,199.9,23.40
4,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,ON06864,06e4e324b83309e575442446e34377d7,1,P02561,1c36a5285f7f3b1ed2637d7c528ce5ff,2017-06-30 02:25:24,99.9,22.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,S1016,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS,ON59939,1ad1ca4550bd6dbd299d8538a86e5400,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-04-10 10:05:10,27.0,16.05
112646,S1016,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS,ON11311,683076b91bcaf79259334f6aa80d41a3,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-02-13 11:32:53,27.0,16.05
112647,S1016,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS,ON17064,aae373a2cc64ed2d1f262aa56e70b7e8,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-02-15 16:42:59,27.0,10.96
112648,S1016,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS,ON41775,d1edb80f6670902ebbb11765b96873da,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-06-26 17:30:16,27.0,12.48


In [35]:
pp =newdf55.drop(['seller_zip_code_prefix', 'seller_city','seller_state'], axis = 1) 

In [37]:
pp.to_csv('olist_order_item_dataset_new.csv')

In [40]:
reviews_db = pd.read_csv("olist_order_reviews_dataset_new.csv")

In [44]:
reviews_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100002 entries, 0 to 100001
Data columns (total 8 columns):
review_id2                 100000 non-null object
review_id                  100002 non-null object
order_id                   100002 non-null object
review_score               100000 non-null float64
review_comment_title       11715 non-null object
review_comment_message     41753 non-null object
review_creation_date       99998 non-null object
review_answer_timestamp    99998 non-null object
dtypes: float64(1), object(7)
memory usage: 6.1+ MB


In [42]:
r1 = orders_db.merge(reviews_db, on='order_id',how='right')

In [43]:
r1[]

,order_id2,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id2,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,ON34305,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,PR31563,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
1,ON12238,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,PR84298,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50
2,ON21442,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,PR41216,e73b67b67587f7644d5bd1a52deb1b01,5.0,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58
3,ON36041,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,PR80287,359d03e676b3c069f62cadba8dd3f6e8,5.0,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03 00:00:00,2017-12-05 19:21:58
4,ON16462,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,PR46847,e50934924e227544ba8246aeb3770dd4,5.0,NaN,NaN,2018-02-17 00:00:00,2018-02-18 13:02:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99997,ON26513,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,PR40080,371579771219f6db2d830d50805977bb,5.0,NaN,Foi entregue antes do prazo.,2017-09-22 00:00:00,2017-09-22 23:10:57
99998,ON14010,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,PR70367,8ab6855b9fe9b812cd03a480a25058a1,2.0,NaN,Foi entregue somente 1. Quero saber do outro p...,2018-01-26 00:00:00,2018-01-27 09:16:56
99999,ON29969,66dea50a8b16d9b4dee7af250b4be1a5,edb027a75a1449115f6b43211ae02a24,delivered,2018-03-08 20:57:30,2018-03-09 11:20:28,2018-03-09 22:11:59,2018-03-16 13:08:30,2018-04-03 00:00:00,PR72262,dc9c59b4688062c25758c2be4cafc523,5.0,NaN,NaN,2018-03-17 00:00:00,2018-03-17 16:33:31
100000,NaN,2017-09-21 12:25:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-21 00:00:00,NaN,NaN,NaN,NaN,NaN


In [48]:
r2 =r1[['review_id2',                
'review_id',                  
'order_id',
 'order_id2',
'review_score',               
'review_comment_title',       
'review_comment_message',     
'review_creation_date',       
'review_answer_timestamp']] 

In [49]:
r2.to_csv('olist_reviews_dataset_new.csv')

In [89]:
orders_db

,order_id2,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,ON34305,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,ON12238,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,ON21442,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,ON36041,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ON16462,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...,...
99436,ON61248,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,ON21623,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,ON26513,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,ON14010,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


In [90]:
customers_db

,customer_id2,uniqueid_2,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,66156,C33212,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,27568,C37191,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,17518,C03781,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,2613,C53477,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,62703,C09832,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...,...,...
99436,31321,C41138,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,63405,C41627,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,7043,C85400,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,15520,C04965,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [92]:
o1 = customers_db.merge(orders_db, on='customer_id',how='right')

In [93]:
o1

,customer_id2,uniqueid_2,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id2,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,66156,C33212,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,ON95945,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00
1,27568,C37191,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,ON56075,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00
2,17518,C03781,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,ON66512,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00
3,2613,C53477,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,ON79913,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00
4,62703,C09832,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,ON00870,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,31321,C41138,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP,ON23820,6760e20addcf0121e9d58f2f1ff14298,delivered,2018-04-07 15:48:17,2018-04-07 16:08:45,2018-04-11 02:08:36,2018-04-13 20:06:37,2018-04-25 00:00:00
99437,63405,C41627,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP,ON18022,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2018-04-04 08:20:22,2018-04-04 08:35:12,2018-04-05 18:42:35,2018-04-11 18:54:45,2018-04-20 00:00:00
99438,7043,C85400,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE,ON64589,fed4434add09a6f332ea398efd656a5c,delivered,2018-04-08 20:11:50,2018-04-08 20:30:03,2018-04-09 17:52:17,2018-05-09 19:03:15,2018-05-02 00:00:00
99439,15520,C04965,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS,ON62932,e31ec91cea1ecf97797787471f98a8c2,delivered,2017-11-03 21:08:33,2017-11-03 21:31:20,2017-11-06 18:24:41,2017-11-16 19:58:39,2017-12-05 00:00:00


In [95]:
o2 =o1.drop(['uniqueid_2', 'customer_unique_id','customer_zip_code_prefix','customer_city','customer_state'], axis = 1) 

In [96]:
o2

,customer_id2,customer_id,order_id2,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,66156,06b8999e2fba1a1fbc88172c00ba8bc7,ON95945,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00
1,27568,18955e83d337fd6b2def6b18a428ac77,ON56075,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00
2,17518,4e7b3e00288586ebd08712fdd0374a03,ON66512,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00
3,2613,b2b6027bc5c5109e529d4dc6358b12c3,ON79913,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00
4,62703,4f2d8ab171c80ec8364f7c12e35b23ad,ON00870,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00
...,...,...,...,...,...,...,...,...,...,...
99436,31321,17ddf5dd5d51696bb3d7c6291687be6f,ON23820,6760e20addcf0121e9d58f2f1ff14298,delivered,2018-04-07 15:48:17,2018-04-07 16:08:45,2018-04-11 02:08:36,2018-04-13 20:06:37,2018-04-25 00:00:00
99437,63405,e7b71a9017aa05c9a7fd292d714858e8,ON18022,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2018-04-04 08:20:22,2018-04-04 08:35:12,2018-04-05 18:42:35,2018-04-11 18:54:45,2018-04-20 00:00:00
99438,7043,5e28dfe12db7fb50a4b2f691faecea5e,ON64589,fed4434add09a6f332ea398efd656a5c,delivered,2018-04-08 20:11:50,2018-04-08 20:30:03,2018-04-09 17:52:17,2018-05-09 19:03:15,2018-05-02 00:00:00
99439,15520,56b18e2166679b8a959d72dd06da27f9,ON62932,e31ec91cea1ecf97797787471f98a8c2,delivered,2017-11-03 21:08:33,2017-11-03 21:31:20,2017-11-06 18:24:41,2017-11-16 19:58:39,2017-12-05 00:00:00


In [97]:
o2.to_csv('olist_orders_dataset_new.csv')

In [38]:
###-------REMOVING EXTRA COLUMNS FROM ALL FILES

In [98]:
customers_db1 = pd.read_csv("olist_customers_dataset_new.csv")
order_items_db1 = pd.read_csv("olist_order_item_dataset_new.csv")
order_payments_db1 = pd.read_csv("olist_order_payments_dataset_new.csv")
orders_db1 = pd.read_csv("olist_orders_dataset_new.csv")
products_db1 = pd.read_csv("olist_products_dataset_new.csv")
sellers_db1 = pd.read_csv("olist_sellers_dataset_new.csv")
reviews_db1 = pd.read_csv("olist_reviews_dataset_new.csv")

In [128]:
customers_db1

,customer_id2,uniqueid_2,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,66156,C33212,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,27568,C37191,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,17518,C03781,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,2613,C53477,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,62703,C09832,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...,...,...
99436,31321,C41138,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,63405,C41627,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,7043,C85400,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,15520,C04965,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [129]:
customers_db1 = customers_db1.rename(columns={'customer_id': 'customer_id3','customer_unique_id':'customer_unique_id_3'})

In [130]:
customers_db1 = customers_db1.rename(columns= {'customer_id2': 'customer_id','uniqueid_2': 'customer_unique_id'})

In [131]:
customers_db1

,customer_id,customer_unique_id,customer_id3,customer_unique_id_3,customer_zip_code_prefix,customer_city,customer_state
0,66156,C33212,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,27568,C37191,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,17518,C03781,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,2613,C53477,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,62703,C09832,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...,...,...
99436,31321,C41138,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,63405,C41627,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,7043,C85400,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,15520,C04965,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [135]:
order_items_db1

,Unnamed: 0,sellerid_2,seller_id,order_id2,order_id,order_item_id,productid_2,product_id,shipping_limit_date,price,freight_value
0,0,S1347,3442f8959a84dea7ee197c632cb2df15,ON52911,4a90af3e85dd563884e2afeab1091394,1,P29061,ffb64e34a37740dafb6c88f1abd1fa61,2017-08-25 20:50:19,106.2,9.56
1,1,S1347,3442f8959a84dea7ee197c632cb2df15,ON47598,bc8a5de6abf5b14f98a6135a7fb46731,2,P31368,325a06bcce0da45b7f4ecf2797dd40e4,2017-09-05 12:50:19,10.8,2.42
2,2,S1347,3442f8959a84dea7ee197c632cb2df15,ON69627,6d953888a914b67350d5bc4d48f2acab,1,P32849,f4621f8ad6f54a2e3c408884068be46d,2017-05-11 16:25:11,101.7,15.92
3,3,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,ON44120,7a95f5626df09176d8823cbce3d9db1e,1,P31854,55103cbbe0851d3a303a022cc4140628,2017-08-17 23:30:18,199.9,23.40
4,4,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,ON06864,06e4e324b83309e575442446e34377d7,1,P02561,1c36a5285f7f3b1ed2637d7c528ce5ff,2017-06-30 02:25:24,99.9,22.70
...,...,...,...,...,...,...,...,...,...,...,...
112645,112645,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON59939,1ad1ca4550bd6dbd299d8538a86e5400,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-04-10 10:05:10,27.0,16.05
112646,112646,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON11311,683076b91bcaf79259334f6aa80d41a3,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-02-13 11:32:53,27.0,16.05
112647,112647,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON17064,aae373a2cc64ed2d1f262aa56e70b7e8,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-02-15 16:42:59,27.0,10.96
112648,112648,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON41775,d1edb80f6670902ebbb11765b96873da,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-06-26 17:30:16,27.0,12.48


In [136]:
orders_items_db1 = order_items_db1.rename(columns={'order_id': 'order_id3','product_id':'product_id_3','seller_id':'seller_id3'})

In [137]:
orders_items_db1

,Unnamed: 0,sellerid_2,seller_id3,order_id2,order_id3,order_item_id,productid_2,product_id_3,shipping_limit_date,price,freight_value
0,0,S1347,3442f8959a84dea7ee197c632cb2df15,ON52911,4a90af3e85dd563884e2afeab1091394,1,P29061,ffb64e34a37740dafb6c88f1abd1fa61,2017-08-25 20:50:19,106.2,9.56
1,1,S1347,3442f8959a84dea7ee197c632cb2df15,ON47598,bc8a5de6abf5b14f98a6135a7fb46731,2,P31368,325a06bcce0da45b7f4ecf2797dd40e4,2017-09-05 12:50:19,10.8,2.42
2,2,S1347,3442f8959a84dea7ee197c632cb2df15,ON69627,6d953888a914b67350d5bc4d48f2acab,1,P32849,f4621f8ad6f54a2e3c408884068be46d,2017-05-11 16:25:11,101.7,15.92
3,3,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,ON44120,7a95f5626df09176d8823cbce3d9db1e,1,P31854,55103cbbe0851d3a303a022cc4140628,2017-08-17 23:30:18,199.9,23.40
4,4,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,ON06864,06e4e324b83309e575442446e34377d7,1,P02561,1c36a5285f7f3b1ed2637d7c528ce5ff,2017-06-30 02:25:24,99.9,22.70
...,...,...,...,...,...,...,...,...,...,...,...
112645,112645,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON59939,1ad1ca4550bd6dbd299d8538a86e5400,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-04-10 10:05:10,27.0,16.05
112646,112646,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON11311,683076b91bcaf79259334f6aa80d41a3,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-02-13 11:32:53,27.0,16.05
112647,112647,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON17064,aae373a2cc64ed2d1f262aa56e70b7e8,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-02-15 16:42:59,27.0,10.96
112648,112648,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON41775,d1edb80f6670902ebbb11765b96873da,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-06-26 17:30:16,27.0,12.48


In [138]:
orders_items_db1 = orders_items_db1.rename(columns= {'order_id2': 'order_id','productid_2': 'product_id','sellerid_2':'seller_id'})

In [139]:
orders_items_db1

,Unnamed: 0,seller_id,seller_id3,order_id,order_id3,order_item_id,product_id,product_id_3,shipping_limit_date,price,freight_value
0,0,S1347,3442f8959a84dea7ee197c632cb2df15,ON52911,4a90af3e85dd563884e2afeab1091394,1,P29061,ffb64e34a37740dafb6c88f1abd1fa61,2017-08-25 20:50:19,106.2,9.56
1,1,S1347,3442f8959a84dea7ee197c632cb2df15,ON47598,bc8a5de6abf5b14f98a6135a7fb46731,2,P31368,325a06bcce0da45b7f4ecf2797dd40e4,2017-09-05 12:50:19,10.8,2.42
2,2,S1347,3442f8959a84dea7ee197c632cb2df15,ON69627,6d953888a914b67350d5bc4d48f2acab,1,P32849,f4621f8ad6f54a2e3c408884068be46d,2017-05-11 16:25:11,101.7,15.92
3,3,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,ON44120,7a95f5626df09176d8823cbce3d9db1e,1,P31854,55103cbbe0851d3a303a022cc4140628,2017-08-17 23:30:18,199.9,23.40
4,4,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,ON06864,06e4e324b83309e575442446e34377d7,1,P02561,1c36a5285f7f3b1ed2637d7c528ce5ff,2017-06-30 02:25:24,99.9,22.70
...,...,...,...,...,...,...,...,...,...,...,...
112645,112645,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON59939,1ad1ca4550bd6dbd299d8538a86e5400,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-04-10 10:05:10,27.0,16.05
112646,112646,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON11311,683076b91bcaf79259334f6aa80d41a3,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-02-13 11:32:53,27.0,16.05
112647,112647,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON17064,aae373a2cc64ed2d1f262aa56e70b7e8,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-02-15 16:42:59,27.0,10.96
112648,112648,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON41775,d1edb80f6670902ebbb11765b96873da,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-06-26 17:30:16,27.0,12.48


In [140]:
orders_items_db1=orders_items_db1.drop(['Unnamed: 0'], axis = 1)
orders_items_db1

,seller_id,seller_id3,order_id,order_id3,order_item_id,product_id,product_id_3,shipping_limit_date,price,freight_value
0,S1347,3442f8959a84dea7ee197c632cb2df15,ON52911,4a90af3e85dd563884e2afeab1091394,1,P29061,ffb64e34a37740dafb6c88f1abd1fa61,2017-08-25 20:50:19,106.2,9.56
1,S1347,3442f8959a84dea7ee197c632cb2df15,ON47598,bc8a5de6abf5b14f98a6135a7fb46731,2,P31368,325a06bcce0da45b7f4ecf2797dd40e4,2017-09-05 12:50:19,10.8,2.42
2,S1347,3442f8959a84dea7ee197c632cb2df15,ON69627,6d953888a914b67350d5bc4d48f2acab,1,P32849,f4621f8ad6f54a2e3c408884068be46d,2017-05-11 16:25:11,101.7,15.92
3,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,ON44120,7a95f5626df09176d8823cbce3d9db1e,1,P31854,55103cbbe0851d3a303a022cc4140628,2017-08-17 23:30:18,199.9,23.40
4,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,ON06864,06e4e324b83309e575442446e34377d7,1,P02561,1c36a5285f7f3b1ed2637d7c528ce5ff,2017-06-30 02:25:24,99.9,22.70
...,...,...,...,...,...,...,...,...,...,...
112645,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON59939,1ad1ca4550bd6dbd299d8538a86e5400,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-04-10 10:05:10,27.0,16.05
112646,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON11311,683076b91bcaf79259334f6aa80d41a3,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-02-13 11:32:53,27.0,16.05
112647,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON17064,aae373a2cc64ed2d1f262aa56e70b7e8,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-02-15 16:42:59,27.0,10.96
112648,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON41775,d1edb80f6670902ebbb11765b96873da,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-06-26 17:30:16,27.0,12.48


In [82]:
order_payments_db1

,Unnamed: 0,order_id2,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,0,ON34305,e481f51cbdc54678b7cc49136f2d6af7,1,credit_card,1,18.12
1,1,ON34305,e481f51cbdc54678b7cc49136f2d6af7,3,voucher,1,2.00
2,2,ON34305,e481f51cbdc54678b7cc49136f2d6af7,2,voucher,1,18.59
3,3,ON12238,53cdb2fc8bc7dce0b6741e2150273451,1,boleto,1,141.46
4,4,ON21442,47770eb9100c2d0c44946d9cf07ec65d,1,credit_card,3,179.12
...,...,...,...,...,...,...,...
103881,103881,ON61248,9c5dedf39a927c1b2549525ed64a053c,1,credit_card,3,85.08
103882,103882,ON21623,63943bddc261676b46f01ca7ac2f7bd8,1,credit_card,3,195.00
103883,103883,ON26513,83c1379a015df1e13d02aae0204711ab,1,credit_card,5,271.01
103884,103884,ON14010,11c177c8e97725db2631073c19f07b62,1,credit_card,4,441.16


In [83]:
orders_payments_db1 = order_payments_db1.rename(columns={'order_id': 'order_id3'})

In [84]:
orders_payments_db1=orders_payments_db1.rename(columns= {'order_id2': 'order_id'})

In [85]:
orders_payments_db1

,Unnamed: 0,order_id,order_id3,payment_sequential,payment_type,payment_installments,payment_value
0,0,ON34305,e481f51cbdc54678b7cc49136f2d6af7,1,credit_card,1,18.12
1,1,ON34305,e481f51cbdc54678b7cc49136f2d6af7,3,voucher,1,2.00
2,2,ON34305,e481f51cbdc54678b7cc49136f2d6af7,2,voucher,1,18.59
3,3,ON12238,53cdb2fc8bc7dce0b6741e2150273451,1,boleto,1,141.46
4,4,ON21442,47770eb9100c2d0c44946d9cf07ec65d,1,credit_card,3,179.12
...,...,...,...,...,...,...,...
103881,103881,ON61248,9c5dedf39a927c1b2549525ed64a053c,1,credit_card,3,85.08
103882,103882,ON21623,63943bddc261676b46f01ca7ac2f7bd8,1,credit_card,3,195.00
103883,103883,ON26513,83c1379a015df1e13d02aae0204711ab,1,credit_card,5,271.01
103884,103884,ON14010,11c177c8e97725db2631073c19f07b62,1,credit_card,4,441.16


In [ ]:
orders_payments_db1=orders_payments_db1.drop(['Unnamed: 0'], axis = 1)


In [125]:
orders_payments_db1

,order_id,order_id3,payment_sequential,payment_type,payment_installments,payment_value
0,ON34305,e481f51cbdc54678b7cc49136f2d6af7,1,credit_card,1,18.12
1,ON34305,e481f51cbdc54678b7cc49136f2d6af7,3,voucher,1,2.00
2,ON34305,e481f51cbdc54678b7cc49136f2d6af7,2,voucher,1,18.59
3,ON12238,53cdb2fc8bc7dce0b6741e2150273451,1,boleto,1,141.46
4,ON21442,47770eb9100c2d0c44946d9cf07ec65d,1,credit_card,3,179.12
...,...,...,...,...,...,...
103881,ON61248,9c5dedf39a927c1b2549525ed64a053c,1,credit_card,3,85.08
103882,ON21623,63943bddc261676b46f01ca7ac2f7bd8,1,credit_card,3,195.00
103883,ON26513,83c1379a015df1e13d02aae0204711ab,1,credit_card,5,271.01
103884,ON14010,11c177c8e97725db2631073c19f07b62,1,credit_card,4,441.16


In [99]:
orders_db1

,Unnamed: 0,customer_id2,customer_id,order_id2,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0,66156,06b8999e2fba1a1fbc88172c00ba8bc7,ON95945,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00
1,1,27568,18955e83d337fd6b2def6b18a428ac77,ON56075,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00
2,2,17518,4e7b3e00288586ebd08712fdd0374a03,ON66512,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00
3,3,2613,b2b6027bc5c5109e529d4dc6358b12c3,ON79913,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00
4,4,62703,4f2d8ab171c80ec8364f7c12e35b23ad,ON00870,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
99436,99436,31321,17ddf5dd5d51696bb3d7c6291687be6f,ON23820,6760e20addcf0121e9d58f2f1ff14298,delivered,2018-04-07 15:48:17,2018-04-07 16:08:45,2018-04-11 02:08:36,2018-04-13 20:06:37,2018-04-25 00:00:00
99437,99437,63405,e7b71a9017aa05c9a7fd292d714858e8,ON18022,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2018-04-04 08:20:22,2018-04-04 08:35:12,2018-04-05 18:42:35,2018-04-11 18:54:45,2018-04-20 00:00:00
99438,99438,7043,5e28dfe12db7fb50a4b2f691faecea5e,ON64589,fed4434add09a6f332ea398efd656a5c,delivered,2018-04-08 20:11:50,2018-04-08 20:30:03,2018-04-09 17:52:17,2018-05-09 19:03:15,2018-05-02 00:00:00
99439,99439,15520,56b18e2166679b8a959d72dd06da27f9,ON62932,e31ec91cea1ecf97797787471f98a8c2,delivered,2017-11-03 21:08:33,2017-11-03 21:31:20,2017-11-06 18:24:41,2017-11-16 19:58:39,2017-12-05 00:00:00


In [100]:
orders_o_db1 = orders_db1.rename(columns={'order_id': 'order_id3','customer_id': 'customer_id3'})

In [101]:
orders_o_db1=orders_o_db1.rename(columns= {'order_id2': 'order_id','customer_id2': 'customer_id'})

In [102]:
orders_o_db1

,Unnamed: 0,customer_id,customer_id3,order_id,order_id3,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0,66156,06b8999e2fba1a1fbc88172c00ba8bc7,ON95945,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00
1,1,27568,18955e83d337fd6b2def6b18a428ac77,ON56075,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00
2,2,17518,4e7b3e00288586ebd08712fdd0374a03,ON66512,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00
3,3,2613,b2b6027bc5c5109e529d4dc6358b12c3,ON79913,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00
4,4,62703,4f2d8ab171c80ec8364f7c12e35b23ad,ON00870,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
99436,99436,31321,17ddf5dd5d51696bb3d7c6291687be6f,ON23820,6760e20addcf0121e9d58f2f1ff14298,delivered,2018-04-07 15:48:17,2018-04-07 16:08:45,2018-04-11 02:08:36,2018-04-13 20:06:37,2018-04-25 00:00:00
99437,99437,63405,e7b71a9017aa05c9a7fd292d714858e8,ON18022,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2018-04-04 08:20:22,2018-04-04 08:35:12,2018-04-05 18:42:35,2018-04-11 18:54:45,2018-04-20 00:00:00
99438,99438,7043,5e28dfe12db7fb50a4b2f691faecea5e,ON64589,fed4434add09a6f332ea398efd656a5c,delivered,2018-04-08 20:11:50,2018-04-08 20:30:03,2018-04-09 17:52:17,2018-05-09 19:03:15,2018-05-02 00:00:00
99439,99439,15520,56b18e2166679b8a959d72dd06da27f9,ON62932,e31ec91cea1ecf97797787471f98a8c2,delivered,2017-11-03 21:08:33,2017-11-03 21:31:20,2017-11-06 18:24:41,2017-11-16 19:58:39,2017-12-05 00:00:00


In [121]:
orders_o_db1=orders_o_db1.drop(['Unnamed: 0'], axis = 1)

In [122]:
orders_o_db1

,customer_id,customer_id3,order_id,order_id3,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,66156,06b8999e2fba1a1fbc88172c00ba8bc7,ON95945,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00
1,27568,18955e83d337fd6b2def6b18a428ac77,ON56075,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00
2,17518,4e7b3e00288586ebd08712fdd0374a03,ON66512,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00
3,2613,b2b6027bc5c5109e529d4dc6358b12c3,ON79913,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00
4,62703,4f2d8ab171c80ec8364f7c12e35b23ad,ON00870,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00
...,...,...,...,...,...,...,...,...,...,...
99436,31321,17ddf5dd5d51696bb3d7c6291687be6f,ON23820,6760e20addcf0121e9d58f2f1ff14298,delivered,2018-04-07 15:48:17,2018-04-07 16:08:45,2018-04-11 02:08:36,2018-04-13 20:06:37,2018-04-25 00:00:00
99437,63405,e7b71a9017aa05c9a7fd292d714858e8,ON18022,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2018-04-04 08:20:22,2018-04-04 08:35:12,2018-04-05 18:42:35,2018-04-11 18:54:45,2018-04-20 00:00:00
99438,7043,5e28dfe12db7fb50a4b2f691faecea5e,ON64589,fed4434add09a6f332ea398efd656a5c,delivered,2018-04-08 20:11:50,2018-04-08 20:30:03,2018-04-09 17:52:17,2018-05-09 19:03:15,2018-05-02 00:00:00
99439,15520,56b18e2166679b8a959d72dd06da27f9,ON62932,e31ec91cea1ecf97797787471f98a8c2,delivered,2017-11-03 21:08:33,2017-11-03 21:31:20,2017-11-06 18:24:41,2017-11-16 19:58:39,2017-12-05 00:00:00


In [103]:
products_db1

,productid_2,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,P23996,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,P26436,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,P05410,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,P03253,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,P21582,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...,...
32946,P12196,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,P18029,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,P07866,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,P27808,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [105]:
product_o_db1 = products_db1.rename(columns={'product_id': 'productid_3'})

In [106]:
product_o_db1=product_o_db1.rename(columns= {'productid_2': 'product_id'})

In [107]:
product_o_db1

,product_id,productid_3,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,P23996,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,P26436,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,P05410,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,P03253,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,P21582,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...,...
32946,P12196,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,P18029,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,P07866,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,P27808,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [108]:
sellers_db1

,sellerid_2,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,S1347,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,S2607,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,S1415,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,S0311,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
...,...,...,...,...,...
3090,S0368,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
3091,S0409,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
3092,S0556,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP
3093,S1016,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS


In [109]:
seller_o_db1 = sellers_db1.rename(columns={'seller_id': 'sellerid_2'})

In [110]:
seller_o_db1=seller_o_db1.rename(columns= {'sellerid_2': 'seller_id'})

In [111]:
seller_o_db1

,seller_id,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,S1347,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,S2607,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,S1415,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,S0311,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
...,...,...,...,...,...
3090,S0368,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
3091,S0409,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
3092,S0556,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP
3093,S1016,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS


In [112]:
reviews_db1

,Unnamed: 0,review_id2,review_id,order_id,order_id2,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,0,PR31563,a54f0611adc9ed256b57ede6b6eb5114,e481f51cbdc54678b7cc49136f2d6af7,ON34305,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
1,1,PR84298,8d5266042046a06655c8db133d120ba5,53cdb2fc8bc7dce0b6741e2150273451,ON12238,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50
2,2,PR41216,e73b67b67587f7644d5bd1a52deb1b01,47770eb9100c2d0c44946d9cf07ec65d,ON21442,5.0,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58
3,3,PR80287,359d03e676b3c069f62cadba8dd3f6e8,949d5b44dbf5de918fe9c16f97b45f8a,ON36041,5.0,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03 00:00:00,2017-12-05 19:21:58
4,4,PR46847,e50934924e227544ba8246aeb3770dd4,ad21c59c0840e6cb83a9ceb5573f8159,ON16462,5.0,NaN,NaN,2018-02-17 00:00:00,2018-02-18 13:02:51
...,...,...,...,...,...,...,...,...,...,...
99997,99997,PR40080,371579771219f6db2d830d50805977bb,83c1379a015df1e13d02aae0204711ab,ON26513,5.0,NaN,Foi entregue antes do prazo.,2017-09-22 00:00:00,2017-09-22 23:10:57
99998,99998,PR70367,8ab6855b9fe9b812cd03a480a25058a1,11c177c8e97725db2631073c19f07b62,ON14010,2.0,NaN,Foi entregue somente 1. Quero saber do outro p...,2018-01-26 00:00:00,2018-01-27 09:16:56
99999,99999,PR72262,dc9c59b4688062c25758c2be4cafc523,66dea50a8b16d9b4dee7af250b4be1a5,ON29969,5.0,NaN,NaN,2018-03-17 00:00:00,2018-03-17 16:33:31
100000,100000,NaN,2017-09-21 00:00:00,2017-09-21 12:25:26,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
rev_db1 = reviews_db1.rename(columns={'review_id':'review_id3','order_id': 'order_id3'})

In [115]:
rev_db1=rev_db1.rename(columns= {'review_id2':'review_id','order_id2': 'order_id'})

In [116]:
rev_db1

,Unnamed: 0,review_id,review_id3,order_id3,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,0,PR31563,a54f0611adc9ed256b57ede6b6eb5114,e481f51cbdc54678b7cc49136f2d6af7,ON34305,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
1,1,PR84298,8d5266042046a06655c8db133d120ba5,53cdb2fc8bc7dce0b6741e2150273451,ON12238,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50
2,2,PR41216,e73b67b67587f7644d5bd1a52deb1b01,47770eb9100c2d0c44946d9cf07ec65d,ON21442,5.0,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58
3,3,PR80287,359d03e676b3c069f62cadba8dd3f6e8,949d5b44dbf5de918fe9c16f97b45f8a,ON36041,5.0,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03 00:00:00,2017-12-05 19:21:58
4,4,PR46847,e50934924e227544ba8246aeb3770dd4,ad21c59c0840e6cb83a9ceb5573f8159,ON16462,5.0,NaN,NaN,2018-02-17 00:00:00,2018-02-18 13:02:51
...,...,...,...,...,...,...,...,...,...,...
99997,99997,PR40080,371579771219f6db2d830d50805977bb,83c1379a015df1e13d02aae0204711ab,ON26513,5.0,NaN,Foi entregue antes do prazo.,2017-09-22 00:00:00,2017-09-22 23:10:57
99998,99998,PR70367,8ab6855b9fe9b812cd03a480a25058a1,11c177c8e97725db2631073c19f07b62,ON14010,2.0,NaN,Foi entregue somente 1. Quero saber do outro p...,2018-01-26 00:00:00,2018-01-27 09:16:56
99999,99999,PR72262,dc9c59b4688062c25758c2be4cafc523,66dea50a8b16d9b4dee7af250b4be1a5,ON29969,5.0,NaN,NaN,2018-03-17 00:00:00,2018-03-17 16:33:31
100000,100000,NaN,2017-09-21 00:00:00,2017-09-21 12:25:26,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
rev_db1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100002 entries, 0 to 100001
Data columns (total 10 columns):
Unnamed: 0                 100002 non-null int64
review_id                  100000 non-null object
review_id3                 100002 non-null object
order_id3                  100002 non-null object
order_id                   100000 non-null object
review_score               100000 non-null float64
review_comment_title       11715 non-null object
review_comment_message     41753 non-null object
review_creation_date       99998 non-null object
review_answer_timestamp    99998 non-null object
dtypes: float64(1), int64(1), object(8)
memory usage: 7.6+ MB


In [144]:
rev_db1=rev_db1.drop(['Unnamed: 0'], axis = 1)

In [149]:
olistall = pd.read_csv('OLIST_ALLDATA.csv')

In [151]:
olistall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116581 entries, 0 to 116580
Data columns (total 54 columns):
Unnamed: 0                       116581 non-null int64
order_id2                        116581 non-null object
order_id                         116581 non-null object
customer_id                      116581 non-null object
order_status                     116581 non-null object
order_purchase_timestamp         116581 non-null object
order_approved_at                116581 non-null object
order_delivered_carrier_date     116581 non-null object
order_delivered_customer_date    116581 non-null object
order_estimated_delivery_date    116581 non-null object
order_item_id                    116581 non-null float64
product_id                       116581 non-null object
seller_id                        116581 non-null object
shipping_limit_date              116581 non-null object
price                            116581 non-null float64
freight_value                    116581 non-null

In [152]:
allo_db1 = olistall.rename(columns={'review_id':'review_id3','order_id': 'order_id3','customer_id':'customer_id3','product_id':'product_id3','seller_id':'seller_id3'
                                    ,'customer_unique_id':'customer_uniqueid3'})

In [154]:
allo_db1 = allo_db1.rename(columns={'order_id2':'order_id','productid_2':'product_id','review_id2':'review_id','uniqueid_2':'customer_unique_id','customer_id2':
                            'customer_id','sellerid_2':'seller_id'})

In [155]:
allo_db1

,Unnamed: 0,order_id,order_id3,customer_id3,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,...,seller_city,seller_state,product_category_name_english,review_response_time,order_purchase_year,order_purchase_month,order_purchase_dayofweek,order_purchase_hour,order_purchase_day,order_purchase_mon
0,0,ON34305,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,...,Maua,SP,housewares,1.0,2017,10,0,10,Mon,Oct
1,1,ON34305,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,...,Maua,SP,housewares,1.0,2017,10,0,10,Mon,Oct
2,2,ON34305,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,...,Maua,SP,housewares,1.0,2017,10,0,10,Mon,Oct
3,3,ON40291,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28 00:00:00,...,Maua,SP,housewares,1.0,2017,8,1,18,Tue,Aug
4,4,ON74313,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15 00:00:00,...,Maua,SP,housewares,0.0,2017,8,2,18,Wed,Aug
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116576,116576,ON07986,442a41b00a8a8bda35511fdf1eb65cc5,e42e2e8642f9c09100b6667e321c2811,invoiced,2018-07-11 22:27:32,2018-07-11 22:45:43,2018-07-13 19:02:36,2018-07-20 20:33:29,2018-07-20 00:00:00,...,Sao Paulo,SP,books imported,2.0,2018,7,2,22,Wed,Jul
116577,116577,ON72724,1dbff949801376b8795a093d31cfe205,0cbd2df3d6e9d8b1eaf5cfefc44886bc,delivered,2018-05-16 14:50:54,2018-05-16 15:18:42,2018-05-18 12:10:00,2018-05-21 17:14:08,2018-05-28 00:00:00,...,Sao Paulo,SP,books imported,0.0,2018,5,2,14,Wed,May
116578,116578,ON15901,6775b950cdc4da0ce39b261032f92c84,c4d135422be0d4333f506acde8af9419,delivered,2018-06-13 12:19:26,2018-06-13 17:39:14,2018-06-15 13:09:00,2018-06-16 17:48:30,2018-06-28 00:00:00,...,Sao Paulo,SP,books imported,0.0,2018,6,2,12,Wed,Jun
116579,116579,ON44476,73d60420cd1a179b2d8887d538efe4c2,65d9213dfa004c17dc126503e106e4a3,delivered,2018-08-07 10:52:58,2018-08-07 11:05:18,2018-08-08 08:31:00,2018-08-22 17:02:23,2018-09-04 00:00:00,...,Rio De Janeiro,RJ,books imported,2.0,2018,8,1,10,Tue,Aug


In [156]:
allo_db1=allo_db1.drop(['Unnamed: 0'], axis = 1)

In [157]:
allo_db1

,order_id,order_id3,customer_id3,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,...,seller_city,seller_state,product_category_name_english,review_response_time,order_purchase_year,order_purchase_month,order_purchase_dayofweek,order_purchase_hour,order_purchase_day,order_purchase_mon
0,ON34305,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,...,Maua,SP,housewares,1.0,2017,10,0,10,Mon,Oct
1,ON34305,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,...,Maua,SP,housewares,1.0,2017,10,0,10,Mon,Oct
2,ON34305,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,...,Maua,SP,housewares,1.0,2017,10,0,10,Mon,Oct
3,ON40291,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28 00:00:00,1.0,...,Maua,SP,housewares,1.0,2017,8,1,18,Tue,Aug
4,ON74313,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15 00:00:00,1.0,...,Maua,SP,housewares,0.0,2017,8,2,18,Wed,Aug
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116576,ON07986,442a41b00a8a8bda35511fdf1eb65cc5,e42e2e8642f9c09100b6667e321c2811,invoiced,2018-07-11 22:27:32,2018-07-11 22:45:43,2018-07-13 19:02:36,2018-07-20 20:33:29,2018-07-20 00:00:00,1.0,...,Sao Paulo,SP,books imported,2.0,2018,7,2,22,Wed,Jul
116577,ON72724,1dbff949801376b8795a093d31cfe205,0cbd2df3d6e9d8b1eaf5cfefc44886bc,delivered,2018-05-16 14:50:54,2018-05-16 15:18:42,2018-05-18 12:10:00,2018-05-21 17:14:08,2018-05-28 00:00:00,1.0,...,Sao Paulo,SP,books imported,0.0,2018,5,2,14,Wed,May
116578,ON15901,6775b950cdc4da0ce39b261032f92c84,c4d135422be0d4333f506acde8af9419,delivered,2018-06-13 12:19:26,2018-06-13 17:39:14,2018-06-15 13:09:00,2018-06-16 17:48:30,2018-06-28 00:00:00,1.0,...,Sao Paulo,SP,books imported,0.0,2018,6,2,12,Wed,Jun
116579,ON44476,73d60420cd1a179b2d8887d538efe4c2,65d9213dfa004c17dc126503e106e4a3,delivered,2018-08-07 10:52:58,2018-08-07 11:05:18,2018-08-08 08:31:00,2018-08-22 17:02:23,2018-09-04 00:00:00,1.0,...,Rio De Janeiro,RJ,books imported,2.0,2018,8,1,10,Tue,Aug


In [159]:
customers_db1.to_csv('olistpre1/olist_customers_dataset_new.csv',index = False)

In [160]:
orders_items_db1.to_csv('olistpre1/olist_items_dataset_new.csv',index = False)

In [169]:
orders_items_db1

,seller_id,seller_id3,order_id,order_id3,order_item_id,product_id,product_id_3,shipping_limit_date,price,freight_value
0,S1347,3442f8959a84dea7ee197c632cb2df15,ON52911,4a90af3e85dd563884e2afeab1091394,1,P29061,ffb64e34a37740dafb6c88f1abd1fa61,2017-08-25 20:50:19,106.2,9.56
1,S1347,3442f8959a84dea7ee197c632cb2df15,ON47598,bc8a5de6abf5b14f98a6135a7fb46731,2,P31368,325a06bcce0da45b7f4ecf2797dd40e4,2017-09-05 12:50:19,10.8,2.42
2,S1347,3442f8959a84dea7ee197c632cb2df15,ON69627,6d953888a914b67350d5bc4d48f2acab,1,P32849,f4621f8ad6f54a2e3c408884068be46d,2017-05-11 16:25:11,101.7,15.92
3,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,ON44120,7a95f5626df09176d8823cbce3d9db1e,1,P31854,55103cbbe0851d3a303a022cc4140628,2017-08-17 23:30:18,199.9,23.40
4,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,ON06864,06e4e324b83309e575442446e34377d7,1,P02561,1c36a5285f7f3b1ed2637d7c528ce5ff,2017-06-30 02:25:24,99.9,22.70
...,...,...,...,...,...,...,...,...,...,...
112645,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON59939,1ad1ca4550bd6dbd299d8538a86e5400,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-04-10 10:05:10,27.0,16.05
112646,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON11311,683076b91bcaf79259334f6aa80d41a3,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-02-13 11:32:53,27.0,16.05
112647,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON17064,aae373a2cc64ed2d1f262aa56e70b7e8,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-02-15 16:42:59,27.0,10.96
112648,S1016,e603cf3fec55f8697c9059638d6c8eb5,ON41775,d1edb80f6670902ebbb11765b96873da,1,P17272,e891d4a9622cae3b9fc2ec558bda155b,2017-06-26 17:30:16,27.0,12.48


In [161]:
orders_payments_db1.to_csv('olistpre1/olist_order_payments_dataset_new.csv',index = False)

In [162]:
rev_db1.to_csv('olistpre1/olist_reviews_dataset_new.csv',index = False)

In [163]:
seller_o_db1.to_csv('olistpre1/olist_sellers_dataset_new.csv',index = False)

In [164]:
product_o_db1.to_csv('olistpre1/olist_products_dataset_new.csv',index = False)

In [165]:
orders_o_db1.to_csv('olistpre1/olist_orders_dataset_new.csv',index = False)

In [166]:
allo_db1.to_csv('olistpre1/OLIST_ALL_DATA.csv',index = False)

In [168]:
order_items_db

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [171]:
order_items_db.loc[order_items_db['price']>=50]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
7,000576fe39319847cbb9d288c5617fa6,1,557d850972a7d6f792fd18ae1400d9b6,5996cddab893a4652a15592fb58ab8db,2018-07-10 12:30:45,810.00,70.75
...,...,...,...,...,...,...,...
112644,fffbee3b5462987e66fb49b1c5411df2,1,6f0169f259bb0ff432bfff7d829b9946,213b25e6f54661939f11710a6fddb871,2018-06-28 09:58:03,119.85,20.03
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95


In [172]:
orders_items_db1.loc[orders_items_db1['price']>=50]

,seller_id,seller_id3,order_id,order_id3,order_item_id,product_id,product_id_3,shipping_limit_date,price,freight_value
0,S1347,3442f8959a84dea7ee197c632cb2df15,ON52911,4a90af3e85dd563884e2afeab1091394,1,P29061,ffb64e34a37740dafb6c88f1abd1fa61,2017-08-25 20:50:19,106.20,9.56
2,S1347,3442f8959a84dea7ee197c632cb2df15,ON69627,6d953888a914b67350d5bc4d48f2acab,1,P32849,f4621f8ad6f54a2e3c408884068be46d,2017-05-11 16:25:11,101.70,15.92
3,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,ON44120,7a95f5626df09176d8823cbce3d9db1e,1,P31854,55103cbbe0851d3a303a022cc4140628,2017-08-17 23:30:18,199.90,23.40
4,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,ON06864,06e4e324b83309e575442446e34377d7,1,P02561,1c36a5285f7f3b1ed2637d7c528ce5ff,2017-06-30 02:25:24,99.90,22.70
5,S1589,d1b65fc7debc3361ea86b5f14c68d2e2,ON66091,035201c3c82a97f8a25dd6bd5244b2d5,1,P02561,1c36a5285f7f3b1ed2637d7c528ce5ff,2017-11-22 04:30:29,89.99,45.09
...,...,...,...,...,...,...,...,...,...,...
112629,S0409,f8201cab383e484733266d1906e2fdfa,ON22186,f13f5fdf45611c82c2e37a20c065ddd1,1,P30946,e526c01c04bb7ef5ba2773baffcc85e9,2017-07-16 21:55:24,79.90,35.88
112630,S0409,f8201cab383e484733266d1906e2fdfa,ON84695,683fd5275f1f6459105a681c65da10c7,1,P30946,e526c01c04bb7ef5ba2773baffcc85e9,2017-06-22 15:30:11,79.90,19.10
112631,S0556,74871d19219c7d518d0090283e03c137,ON74732,deea565ebe3196a7408232c8bb0ddc0b,1,P06156,dab1da142d5a95b9200cef4675a3f9fa,2018-07-27 09:19:12,121.03,11.87
112632,S0556,74871d19219c7d518d0090283e03c137,ON62863,4dee2a3e6470bab984859192120d6e3e,1,P15191,5101eb75c72eba49b6ab5e3cb721c99b,2018-05-09 13:11:56,75.90,23.07
